In [ ]:
from src.travel_time_computation.prepare_data import prepare_data_Baltimore
import pandas as pd

block_group_shapefile_path = '../../raw_data/block_group/tl_2020_24_tabblock20.shp'
md_rac_path = '../../raw_data/md_rac_S000_JT00_2020.csv.gz'
md_wac_path = '../../raw_data/md_wac_S000_JT00_2020.csv.gz'

md_rac_df, md_wac_df = prepare_data_Baltimore(block_group_shapefile_path, md_rac_path, md_wac_path)

In [ ]:
import pandas as pd

median_accessibility = pd.read_csv('median_accessibility.csv')

In [ ]:
merged_df = median_accessibility.merge(md_rac_df, left_on = 'h_geocode', right_on = 'id')

In [ ]:
def extract_tract(GEOID):
    GEOID = str(GEOID)
    return GEOID[:11]

merged_df['tract'] = merged_df['h_geocode'].apply(extract_tract)

In [ ]:
new_df = merged_df['gravity'].groupby(merged_df['tract']).mean() / merged_df['C000'].groupby(merged_df['tract']).sum()

In [ ]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from census import Census
from us import states

c = Census("93c3297165ad8b5b6c81e0ed9e2e44a38e56224f")

md_tract = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2022/TRACT/tl_2022_24_tract.zip")


In [ ]:

# Reproject shapefile to UTM Zone 17N
# https://spatialreference.org/ref/epsg/wgs-84-utm-zone-17n/
md_tract = md_tract.to_crs(epsg = 32617)

# Print GeoDataFrame of shapefile
print(md_tract.head(2))

In [ ]:
# convert new_df to dataframe and change index into a column
new_df = new_df.reset_index()
new_df

In [ ]:
# change column name from 0 to access_by_pop
new_df = new_df.rename(columns = {0: 'access_by_pop'})

In [ ]:
new_df

In [ ]:
final = md_tract.merge(new_df, left_on = 'GEOID', right_on = 'tract')

In [ ]:
final

In [ ]:
final['access_by_pop'][final['access_by_pop'] > 50] = 50

In [ ]:
final.explore(column = 'access_by_pop', cmap='Greens',
                style_kwds=dict(color="black",weight=1, opacity=0.3, fillOpacity=0.8, edgecolor='none'))